**Question 6**
Student : Arora, Sanjana (V00966221)

** Q6.1. Predicting poverty level using decision regressor **

In [ ]:
import numpy as np
import pandas as pd
import sklearn

Please download election_clean file provided along with the zipped folder

In [ ]:
df = pd.read_csv('elections_clean.csv')

In [ ]:
df.columns

Index(['Unnamed: 0', 'votes', 'UnemploymentRate2015', 'MedHHInc2014',
       'PerCapitaInc', 'PovertyAllAgesPct2014', 'Deep_Pov_All', 'Population',
       'Area in square miles - Total area', 'PopDensity', 'TOT_MALE_rate',
       'TOT_FEMALE_rate', 'voter_turnout_rate', 'Democrat', 'State', 'County',
       'Education', 'Religion', 'Old', 'Young', 'Adult', 'EthnicMale',
       'EthnicFemale'],
      dtype='object')

In [ ]:
# Creating subset of dataframes capturing desired features for analysis
subset = df[['Education', 'Religion', 'EthnicMale', 'EthnicFemale','PerCapitaInc']].copy()

In [ ]:
# Transforming the categorical features - percapita feature is excluded as it is continuous
subset_transform = pd.get_dummies(subset[['Education', 'Religion', 'EthnicMale', 'EthnicFemale']])

In [ ]:
# making the combined dataset including the percapita
X = pd.concat([subset_transform, df['PerCapitaInc']], axis=1)

In [ ]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

In [ ]:
# Target Feature
Y = df['PovertyAllAgesPct2014']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3, shuffle= True)

In [ ]:
tree_reg = tree.DecisionTreeRegressor(random_state=42)

In [ ]:
tree_reg = tree_reg.fit(X_train, y_train)

In [ ]:
Y_pred = tree_reg.predict(X_train)

In [ ]:
print('The training Mean Absolute Error is:', mean_absolute_error(y_train,Y_pred))

The training Mean Absolute Error is: 0.0007914584279872785


In [ ]:
Y_pred1 = tree_reg.predict(X_test)

In [ ]:
print('The testing Mean Absolute Error is:', mean_absolute_error(y_test,Y_pred1))

The testing Mean Absolute Error is: 0.03526271186440678


Q6.2 **5 Fold Cross Validation from Scratch**

In [ ]:
# Splitting the dataset into training and testing dataset
def train_validation_split(dataset):
    validation_split = .3
    dataset_size = len(dataset)
    split = int(np.floor(validation_split * dataset_size))
    training_data = dataset.iloc[split:].reset_index(drop=True)
    validation_data = dataset.iloc[:split].reset_index(drop=True)
    return training_data,validation_data

In [ ]:
# Subset to be considered for analysis
subset2 = pd.concat([subset_transform, df['PerCapitaInc'], df['PovertyAllAgesPct2014']], axis=1)
training_data,testing_data = train_validation_split(subset2)

In [ ]:
training_data.columns

Index(['Education_BachelorOrHigher', 'Education_College',
       'Education_LessthanDiploma', 'Education_OnlyDiploma', 'Religion_Amish',
       'Religion_Catholic', 'Religion_Christian Generic', 'Religion_Mormon',
       'Religion_Other', 'Religion_Other Misc', 'EthnicMale_ASIAN_MALE_rate',
       'EthnicMale_BLACK_MALE_rate', 'EthnicMale_NATIVE_AMERICAN_MALE_rate',
       'EthnicMale_WHITE_MALE_rate', 'EthnicFemale_BLACK_FEMALE_rate',
       'EthnicFemale_NATIVE_AMERICAN_FEMALE_rate',
       'EthnicFemale_WHITE_FEMALE_rate', 'PerCapitaInc',
       'PovertyAllAgesPct2014'],
      dtype='object')

In [ ]:
# function for creating indices for n folds of dataset
def generate_folds(df, n):
    
    #Getting a randmly shuffled index array of the rows of dataframe  
    split=np.random.choice(a = int(df.shape[0]), size=int(df.shape[0])) 
    
    #Splitting the index array into n arrays
    #Storing the random index of the n folds dataframes
    five_split = np.array_split(split, n)
    # creating empty list to store dataframes
    fold =[]
    #Creating the n folds
    for i in range(n):
    
      df_fold = df.iloc[five_split[i]] 
      fold.append(df_fold)
    return fold
    

Function for implementing five fold cross-validation and training the regression tree and evaluating the error metrics. The function takes the following inputs:
1. Training Data
2. Target Name
3. The depth of the regression trees

In [ ]:
def Five_FoldCV_And_ModelEvaluation(training_data,Target_Name, depth_level, n_folds):
    
    #Creating the Five Fold Dataframes from the list
    Z=generate_folds(training_data,n_folds)
    df1 = pd.DataFrame(Z[1])
    df2 = pd.DataFrame(Z[2])
    df3 = pd.DataFrame(Z[3])
    df4 = pd.DataFrame(Z[4])
    df5 = pd.DataFrame(Z[0])
    
    from sklearn.tree import DecisionTreeRegressor
   
    # list capturing the errors at each fold
    error_list=[]
    
    for i in range(1,6):
        
        if(i==1):
            # When i = 1 , making the fifth set as the test set
            # And the rest of the sets are merged to be used for training
            f5 = df5
            
            #Merging the rest of the dataframes
            Tmerge=pd.concat([df1,df2,df3,df4],ignore_index=True)
            
            #Creating target vector from the merged dataframe
            train_y= Tmerge[Target_Name]
            #Feature sets from merged dataframe
            train_X= Tmerge.drop(columns=Target_Name,axis=1)
            
            #Creating target vector from the test dataframe
            test_y = f5[Target_Name]
            #Feature sets from test dataframe
            test_X = f5.drop(columns=Target_Name,axis=1)
            
            #Model training and evaluation
            model_1=DecisionTreeRegressor(max_depth=depth_level)
            model_1.fit(train_X,train_y)
            predictions_1 = model_1.predict(test_X)
            mae_1 = mean_absolute_error(test_y,predictions_1)
            
            #Storing the Error result in the array
            error_list.append(mae_1)
            
        elif(i==2):
            # When i = 1 , making the fourth set as the test set
            # And the rest of the sets are merged to be used for training
            f4=df4
            #Merging the rest of the dataframes
            Tmerge=pd.concat([df1,df2,df3,df5],ignore_index=True)
            
            #Creating target vector from the merged dataframe
            train_y= Tmerge[Target_Name]
            #Feature sets from merged dataframe
            train_X= Tmerge.drop(columns=Target_Name,axis=1)
            
            #Creating target vector from the test dataframe
            test_y = f4[Target_Name]
            #Feature sets from test dataframe
            test_X = f4.drop(columns=Target_Name,axis=1)
            
            #Model training and evaluation
            model_2=DecisionTreeRegressor(max_depth=depth_level)
            model_2.fit(train_X,train_y)
            predictions_2 = model_2.predict(test_X)
            mae_2 = mean_absolute_error(test_y,predictions_2)
            
            #Storing the Error result in the array
            error_list.append(mae_2)
            
        elif(i==3):
            # When i = 1 , making the third set as the test set
            # And the rest of the sets are merged to be used for training
            f3=df3
            #Merging the rest of the dataframes
            Tmerge=pd.concat([df1,df2,df4,df5],ignore_index=True)
            
            #Creating target vector from the merged dataframe
            train_y= Tmerge[Target_Name]
            #Feature sets from merged dataframe
            train_X= Tmerge.drop(columns=Target_Name,axis=1)
            
           #Creating target vector from the test dataframe
            test_y = f3[Target_Name]
            #Feature sets from test dataframe
            test_X = f3.drop(columns=Target_Name,axis=1)
            
            #Model training and evaluation
            model_3=DecisionTreeRegressor(max_depth=depth_level)
            model_3.fit(train_X,train_y)
            predictions_3 = model_3.predict(test_X)
            mae_3 = mean_absolute_error(test_y,predictions_3)
            
            #Storing the Error result in the array
            error_list.append(mae_3)
        
        elif(i==4):
            # # When i = 1 , making the second set as the test set
            # And the rest of the sets are merged to be used for training
            f2=df2
            #Merging the rest of the dataframes
            fold_merge=pd.concat([df1,df3,df4,df5],ignore_index=True)
            
            #Creating target vector from the merged dataframe
            train_y= Tmerge[Target_Name]
           #Feature sets from test dataframe
            train_X= Tmerge.drop(columns=Target_Name,axis=1)
            
            #Creating target vector from the test dataframe
            test_y = f2[Target_Name]
            #Creating X_test from test fold
            test_X = f2.drop(columns=Target_Name,axis=1)
            
            #Model training and evaluation
            model_4=DecisionTreeRegressor(max_depth=depth_level)
            model_4.fit(train_X,train_y)
            predictions_4 = model_4.predict(test_X)
            mae_4 = mean_absolute_error(test_y,predictions_4)
            
            #Storing the Error result in the array
            error_list.append(mae_4)
    
        elif(i==5):
            # # When i = 1 , making the first set as the test set
            # And the rest of the sets are merged to be used for training
            f1=df1
           #Merging the rest of the dataframes
            fold_merge=pd.concat([df2,df3,df4,df5],ignore_index=True)
            
            #Creating target vector from the merged dataframe
            train_y= Tmerge[Target_Name]
            #Feature sets from merged dataframe
            train_X= Tmerge.drop(columns=Target_Name,axis=1)
            
            #Creating target vector from the test dataframe
            test_y = f1[Target_Name]
            #Feature sets from test dataframe
            test_X = f1.drop(columns=Target_Name,axis=1)
            
            #Model training and evaluation
            model_5=DecisionTreeRegressor(max_depth=depth_level)
            model_5.fit(train_X,train_y)
            predictions_5 = model_5.predict(test_X)
            mae_5 = mean_absolute_error(test_y,predictions_5)
            
            #Storing the Error result in the array
            error_list.append(mae_5)
    
    #calculating the final error by averaging all the errors
    finalError=sum(error_list)/len(error_list)
    return finalError, error_list

In [ ]:
# Calculating the metrics by changing the depth of the trees
# Considering depth = 5
result,error_list = Five_FoldCV_And_ModelEvaluation(training_data,'PovertyAllAgesPct2014',5,5)
print(result,error_list)

0.02364809832545115 [0.024954431108903153, 0.02333241227138217, 0.024147091000610545, 0.023687591297824807, 0.022118965948535074]


In [ ]:
# Considering depth = 3
result,error_list = Five_FoldCV_And_ModelEvaluation(training_data,'PovertyAllAgesPct2014',3,5)
print(result,error_list)

0.026664994304767077 [0.026147782892587783, 0.027361346558500626, 0.02659574023863196, 0.025696225200869158, 0.02752387663324586]


In [ ]:
# considering depth = 7
result,error_list = Five_FoldCV_And_ModelEvaluation(training_data,'PovertyAllAgesPct2014',7,5)
print(result,error_list)

0.020545218735925387 [0.02080871553798112, 0.02226819678700516, 0.02312959277208091, 0.016873431387443783, 0.01964615719511597]


From the above mean absolute error results on the testing sets using each of the different depth of regression trees, we observe that the MAE average error for testing data is the least for the regression tree with length = 7. Therefore, out of the three depths of the tree considered, the regression tree with depth 7 is the best.  